# Resampling

The next idea I would like to demonstrate is resampling. This is a method of using a dataset to simmulate the random sampling process on the population in order to obtain estimates for a statistic and its possible varriation. You will see that it is a process that is designed to perform well with larger samples, though truthfully it yields valid results (though with caveats) in any case. It cannot however fix a sampling process that is flawed.

---

## Bootstrapping

Is the specific case of rebuilding a sample of exactly the same size as the original through resampling with replacement.

## Procedure

The procedure is to build a new sample by sampling from the dataset with replacement. One then computes the statistic of interest from the new sample and records the value. This process is then repeated multiple times. The result range of computed values forms an analogue of the confidence interval, and one can then reproduce hypothesis tests as well. 

## Simulating Data

A Big issue with educational (and other types of data) is that it is often protected under stringent protocols. A method developed by the Health Insurance industry is effective in creating a data set that merely simmulates the original dataset and in particular does not include the record of any indvidual and so can be used in ways that the original data could not be. One can do this by resampling with replacement from the original data forcing it to fit the parameters of a model to preserve what you think would be interesting features (covariances for instance). 

It is primarily a tool for when the code for a study is in development allowing one to study the analysis process and tools without having to be as careful with the protected data, with the idea that after things are developed we would then run the code for real on the protected data to obtain the final parameters and models.



In [ ]:
penguins = read.csv("Datasets/penguins_lter.csv")
gentoo = penguins[penguins[,3]=='Gentoo penguin (Pygoscelis papua)',]
gentoo_flipper = na.omit(gentoo[,12])
# Note I had to add an na.omit

length(gentoo_flipper)

In [ ]:
# Let's bootstrap this dataset a bunch of times and compute the mean each time:

data = c()
for (i in 1:50) { 
  temp = sample(gentoo_flipper, 123, replace = TRUE)
  data = c(data, mean(temp))
}

data

In [ ]:
hist(data)

In [ ]:
boxplot(data)

# The Datascience Modeling Process

Let's return to the classification of penguins problem. A problem confronting us is to choose a model that gives the *best* prediction of the species of penguins from the inputs that we can manage. More than that, what we want to understand about the model we decide on is what we expect its error to be. 

Note the danger:  As we increase the complexity of our model we obtain a result which adapts more to the training data. However the training data began as a random sample from a population - and so it contains irreducible errors caused by factors we do not (and in most cases cannot hope to) know.  If our model moves too closely to the training data, it may be responding to these errors rather than actual structure in the population - and we would expect this to then lead to a higher error rate when the model is applied to data it has never seen: the testing set or even worse, new observations. 

This is called **Overfitting**. 

Accordingly, much of our effort is going to go into overfitting. We will control for overfitting by repeatedly in the exploration phase of our analysis applying the models we have develop to new testing data that the model has not yet seen. In real applications, with *plenty* of data this process might be nested in multiple layers with a final testing set that is reserved until the final model has been selected and trained. 

The process of training and testing our model with multiple slices of our dataset is called **Crossvalidation**. And the goal is to find a model that captures true features in the data, avoids the overfitting that arises from responding to irreducible errors in the dataset, and to have a good estimate on the error and its varriation we expect from the final model. 

## Ethics

Note that how much effort one actually puts into this crossvalidation process would be determined by the ethical consideration of how likely errors seem to be, and on what the consequences for errors are. There is a big difference between us playing around with a penguin dataset, and us working for a university on student success data, financial data, or health data. 

## Cross Validation

There are lots of ways to do this. However a common one is to divide the data set randomly into 5 equal size pieces. We will then train our model on four of these pieces using the fifth one as a testing set. We can then repeate that with the same model and parameters five times obtaining an error estimate each time.

This will give us five estimates on the error produced by training the model with these parameters on a random training set when the model is applied to data it has not seen. Each individual data point is used four times as a training value and once as a testing value. 

In [ ]:
# we need to remove the rows with missing data; and also pair the data down to just the values we will use
data = penguins[,c(3, 12,11)]
data = na.omit(data)
data$Species = as.factor(data$Species)   # we are going to use decision trees below, and for them the output column needs to be a factor and the inputs numerical or factors.
head(data)

In [ ]:
x = sample(nrow(data), nrow(data) )   # Shuffle the row indexes
splits = split(x, as.integer((x - 1)/(nrow(data)*0.2)) )   # divide the row indexes up into 5 mostly equal sized pieces
shuffles = list()

for (i in 1:5) {
    shuffles[[i]] = sort(x[splits[[i]]])
}

In [ ]:
test = list()
train = list()

for (i in 1:5) {
    test[[i]] = data[shuffles[[i]],]
    train[[i]] = data[-shuffles[[i]], ]
}

## K Nearest Neighbors Again

We can now test this with the k-nearest neighbors algorithm we met last week.

In [ ]:
library(class)

In [ ]:
out = c()

for (i in 1:5) { 
    pr = knn(train[[i]][,-1],test[[i]][,-1],cl=train[[i]][,1],k=1)
    out = c(out, sum(test[[i]][,1]!=pr)/length(test[[i]][,1]))
}


In [ ]:
out
mean(out)
sd(out)

## Decision Trees

It's worth looking at some other models now as well. One of the simplest models we can build is called a decision tree. Decision trees are a list of rules to mkae our classification.

They are even easier to explain than the KNN algorithm. And we see examples of them all around us:  For example the placement mechanism UNC uses for our courses is a decision tree: https://www.unco.edu/nhs/mathematical-sciences/placement/results.aspx

which tries to classify students into their mathemaitcs courses.

In [ ]:
library(tree)
library(rpart)
library(rpart.plot)

In [ ]:
md <- rpart(Species ~ Flipper.Length..mm. + Culmen.Depth..mm., data=train[[1]] )
md

In [ ]:
pdf("plot_tree.pdf") 
rpart.plot(md, box.palette="RdBu", shadow.col="gray", nn=TRUE )
dev.off() 

In [ ]:
help(predict.rpart)